In [32]:
import bs4 as bs
import requests
import datetime
import yfinance as yf
import pandas as pd

In [35]:
class SP500DataDownloader:
    def __init__(self, start_date, end_date, output_file='./data/sp500_historical_data.csv'):
        self.start_date = start_date
        self.end_date = end_date
        self.output_file = output_file
        self.tickers = []

    def scrape_sp500_tickers(self):
        """Scrape the S&P 500 tickers from Wikipedia."""
        print("Scraping S&P 500 tickers...")
        tickers_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
        self.tickers = tickers_df['Symbol'].tolist()
        print(f"Scraped {len(self.tickers)} tickers.")
        return self.tickers

    def download_stock_data(self):
        """Download historical stock data for the S&P 500 tickers and write it to the output file."""
        # If tickers have not been scraped yet, run the scrape method
        if not self.tickers:
            self.scrape_sp500_tickers()

        all_stock_data = pd.DataFrame()

        # Iterate over each symbol and download the stock data
        for symbol in self.tickers:
            print(f"Downloading data for {symbol}...")

            # Download the stock data
            stock_data = yf.download(symbol, start=self.start_date, end=self.end_date)
            
            # Check if stock_data is empty
            if stock_data.empty:
                print(f"No data found for {symbol}. Skipping...")
                continue
            
            # Add the ticker symbol as a column to the DataFrame
            stock_data['Symbol'] = symbol
            
            # Append the stock data to the all_stock_data DataFrame
            all_stock_data = pd.concat([all_stock_data, stock_data])

        # Reset index and save the DataFrame to a CSV file
        if not all_stock_data.empty:
            all_stock_data.reset_index(inplace=True)
            all_stock_data.to_csv(self.output_file, index=False)
            print(f"Stock data saved to {self.output_file}")
        else:
            print("No stock data to save.")

        return all_stock_data

In [39]:
start_date = '2018-01-01'
end_date = '2024-08-01'
output_file = '../data/sp500_historical_data.csv'

# Initialize the SP500DataDownloader class
downloader = SP500DataDownloader(start_date=start_date, end_date=end_date, output_file=output_file)
df = downloader.download_stock_data()

Scraping S&P 500 tickers...
Scraped 503 tickers.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Stock data saved to ../data/sp500_historical_data.csv
